# Gesture Recognition
- Shubhi Shukla
- Kshitij Raj

In [1]:
import numpy as np
import os
import imageio
import datetime
import os
import tensorflow as tf

from imageio import imread
from skimage.transform import resize
import datetime
import os

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 8

## Generator

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =  [x for x in range(0,30)]  
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size    
        for batch in range(num_batches):     
            batch_data = np.zeros((batch_size,len(img_idx),120,120,3))   
            batch_labels = np.zeros((batch_size,5))                                
            for folder in range(batch_size):                                         
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
                for idx,item in enumerate(img_idx):         
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image= resize(image,[120,120])
                    
                    
                    batch_data[folder,idx,:,:,0] = (image[:, : , 0] - np.percentile(image[:, : , 0],5))/ (np.percentile(image[:, : , 0],95) - np.percentile(image[:, : , 0],5))
                    batch_data[folder,idx,:,:,1] = (image[:, : , 1] - np.percentile(image[:, : , 1],5))/ (np.percentile(image[:, : , 1],95) - np.percentile(image[:, : , 1],5))
                    batch_data[folder,idx,:,:,2] = (image[:, : , 2] - np.percentile(image[:, : , 2],5))/ (np.percentile(image[:, : , 2],95) - np.percentile(image[:, : , 2],5))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels 

        
        if (len(folder_list))> batch_size*num_batches:
            batch=(len(folder_list) // batch_size)
            remaining_folders = len(folder_list)- ((len(folder_list) // batch_size)*batch_size)   
            batch_data = np.zeros((remaining_folders,len(img_idx),120,120,3)) 
            batch_labels = np.zeros((remaining_folders,5)) 
            for folder in range(remaining_folders): 
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
                for idx,item in enumerate(img_idx): 
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image= resize(image,[120,120])
                    
                    
                    batch_data[folder,idx,:,:,0] = (image[:, : , 0] - np.percentile(image[:, : , 0],5))/ (np.percentile(image[:, : , 0],95) - np.percentile(image[:, : , 0],5))
                    batch_data[folder,idx,:,:,1] = (image[:, : , 1] - np.percentile(image[:, : , 1],5))/ (np.percentile(image[:, : , 1],95) - np.percentile(image[:, : , 1],5))
                    batch_data[folder,idx,:,:,2] = (image[:, : , 2] - np.percentile(image[:, : , 2],5))/ (np.percentile(image[:, : , 2],95) - np.percentile(image[:, : , 2],5))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20  # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model - Conv3D

In [6]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
#from keras.optimizers import Adam

### Model 1

In [7]:
Input_shape = (30, 120, 120, 3)
model = Sequential()
model.add(Conv3D(32, (2,2,2),input_shape=Input_shape,activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (2,2,2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
#model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(256,activation='relu'))
#model.add(Activation('relu'))
model.add(Dropout(0.4))


model.add(Dense(5))
model.add(Activation('softmax'))

In [8]:
model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 29, 119, 119, 32)  800       
_________________________________________________________________
batch_normalization (BatchNo (None, 29, 119, 119, 32)  128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 14, 59, 59, 32)    0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 59, 59, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 13, 58, 58, 64)    16448     
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 58, 58, 64)    256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 29, 29, 64)     0

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
batch_size

8

In [13]:
Input_shape

(30, 120, 120, 3)

In [15]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  /home/datasets/Project_data/train ; batch size = 8
Epoch 1/20
83/83 [==============================] - ETA: 0s - loss: 48.6160 - categorical_accuracy: 0.2172Source path =  /home/datasets/Project_data/val ; batch size = 8

Epoch 00001: saving model to model_init_2021-10-2614_54_40.749188/model-00001-48.61595-0.21719-72.57760-0.18000.h5
83/83 [==============================] - 220s 3s/step - loss: 48.6160 - categorical_accuracy: 0.2172 - val_loss: 72.5776 - val_categorical_accuracy: 0.1800
Epoch 2/20
83/83 [==============================] - ETA: 0s - loss: 4.6082 - categorical_accuracy: 0.2323
Epoch 00002: saving model to model_init_2021-10-2614_54_40.749188/model-00002-4.60823-0.23228-112.04321-0.17000.h5
83/83 [==============================] - 222s 3s/step - loss: 4.6082 - categorical_accuracy: 0.2323 - val_loss: 112.0432 - val_categorical_accuracy: 0.1700
Epoch 3/20
83/83 [=====================

### Model 2

In [16]:
batch_size = 16
Input_shape = (30, 120, 120, 3)
model2 = Sequential()
model2.add(Conv3D(32, (2,2,2),input_shape=Input_shape,activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size=(2,2,2)))
model2.add(Dropout(0.25))

model2.add(Conv3D(64, (2,2,2),activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size=(2,2,2)))
model2.add(Dropout(0.25))

model2.add(Flatten())

model2.add(Dense(128,activation='relu'))
#model.add(Activation('relu'))
model2.add(Dropout(0.4))

model2.add(Dense(256,activation='relu'))
#model.add(Activation('relu'))
model2.add(Dropout(0.4))


model2.add(Dense(5))
model2.add(Activation('softmax'))

model2.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 29, 119, 119, 32)  800       
_________________________________________________________________
batch_normalization_2 (Batch (None, 29, 119, 119, 32)  128       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 14, 59, 59, 32)    0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 59, 59, 32)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 13, 58, 58, 64)    16448     
_________________________________________________________________
batch_normalization_3 (Batch (None, 13, 58, 58, 64)    256       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 6, 29, 29, 64)    

In [17]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [18]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 16
Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 29.7599 - categorical_accuracy: 0.2338Source path =  /home/datasets/Project_data/val ; batch size = 16

Epoch 00001: saving model to model_init_2021-10-2614_54_40.749188/model-00001-29.75992-0.23379-3.47802-0.27000.h5
42/42 [==============================] - 202s 5s/step - loss: 29.7599 - categorical_accuracy: 0.2338 - val_loss: 3.4780 - val_categorical_accuracy: 0.2700
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 2.8755 - categorical_accuracy: 0.2398
Epoch 00002: saving model to model_init_2021-10-2614_54_40.749188/model-00002-2.87548-0.23982-73.13662-0.23000.h5
42/42 [==============================] - 201s 5s/step - loss: 2.8755 - categorical_accuracy: 0.2398 - val_loss: 73.1366 - val_categorical_accuracy: 0.2300
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 2.0097 - categorical_accuracy: 0.2338
Epoch 00003

### Model 3

In [19]:
batch_size = 32
Input_shape = (30, 120, 120, 3)
model3 = Sequential()
model3.add(Conv3D(32, (2,2,2),input_shape=Input_shape,activation='relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,2)))
model3.add(Dropout(0.25))

model3.add(Conv3D(64, (2,2,2),activation='relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,2)))
model3.add(Dropout(0.25))

model3.add(Conv3D(128, (2,2,2),activation='relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,2)))
model3.add(Dropout(0.25))

model3.add(Flatten())

model3.add(Dense(256,activation='relu'))
#model.add(Activation('relu'))
model3.add(Dropout(0.4))


model3.add(Dense(5))
model3.add(Activation('softmax'))

model3.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 29, 119, 119, 32)  800       
_________________________________________________________________
batch_normalization_4 (Batch (None, 29, 119, 119, 32)  128       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 14, 59, 59, 32)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 59, 59, 32)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 13, 58, 58, 64)    16448     
_________________________________________________________________
batch_normalization_5 (Batch (None, 13, 58, 58, 64)    256       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 6, 29, 29, 64)    

In [20]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [21]:
model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 32.8031 - categorical_accuracy: 0.2640Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: val_loss improved from inf to 2.29374, saving model to model_init_2021-10-2614_54_40.749188/model-00001-32.80305-0.26395-2.29374-0.30000.h5
21/21 [==============================] - 221s 11s/step - loss: 32.8031 - categorical_accuracy: 0.2640 - val_loss: 2.2937 - val_categorical_accuracy: 0.3000
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 1.8860 - categorical_accuracy: 0.3288
Epoch 00002: val_loss did not improve from 2.29374
21/21 [==============================] - 213s 10s/step - loss: 1.8860 - categorical_accuracy: 0.3288 - val_loss: 56.2931 - val_categorical_accuracy: 0.3000
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 1.5224 - categorical_accuracy: 0.3560
Epoch 00003: val_loss did not imp

### Model 4

In [22]:
batch_size = 64
Input_shape = (30, 120, 120, 3)
model4 = Sequential()
model4.add(Conv3D(16, (2,2,2),input_shape=Input_shape,activation='relu'))
model4.add(BatchNormalization())
model4.add(MaxPooling3D(pool_size=(2,2,2)))
model4.add(Dropout(0.25))

model4.add(Conv3D(32, (2,2,2),activation='relu'))
model4.add(BatchNormalization())
model4.add(MaxPooling3D(pool_size=(2,2,2)))
model4.add(Dropout(0.25))

model4.add(Conv3D(64, (2,2,2),activation='relu'))
model4.add(BatchNormalization())
model4.add(MaxPooling3D(pool_size=(2,2,2)))
model4.add(Dropout(0.25))

model4.add(Flatten())

model4.add(Dense(256,activation='relu'))
#model.add(Activation('relu'))
model4.add(Dropout(0.4))


model4.add(Dense(5))
model4.add(Activation('softmax'))

model4.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model4.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_26 (Conv3D)           (None, 29, 119, 119, 16)  400       
_________________________________________________________________
batch_normalization_26 (Batc (None, 29, 119, 119, 16)  64        
_________________________________________________________________
max_pooling3d_26 (MaxPooling (None, 14, 59, 59, 16)    0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 14, 59, 59, 16)    0         
_________________________________________________________________
conv3d_27 (Conv3D)           (None, 13, 58, 58, 32)    4128      
_________________________________________________________________
batch_normalization_27 (Batc (None, 13, 58, 58, 32)    128       
_________________________________________________________________
max_pooling3d_27 (MaxPooling (None, 6, 29, 29, 32)    

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [24]:
model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/20
11/11 [==============================] - ETA: 0s - loss: 24.3242 - categorical_accuracy: 0.2293 Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: val_loss improved from inf to 1.67214, saving model to model_init_2021-10-2704_53_32.764571/model-00001-24.32424-0.22926-1.67214-0.31000.h5
11/11 [==============================] - 203s 18s/step - loss: 24.3242 - categorical_accuracy: 0.2293 - val_loss: 1.6721 - val_categorical_accuracy: 0.3100
Epoch 2/20
11/11 [==============================] - ETA: 0s - loss: 3.1658 - categorical_accuracy: 0.4314 
Epoch 00002: val_loss did not improve from 1.67214
11/11 [==============================] - 202s 18s/step - loss: 3.1658 - categorical_accuracy: 0.4314 - val_loss: 7.4262 - val_categorical_accuracy: 0.2700
Epoch 3/20
11/11 [==============================] - ETA: 0s - 

### Model 5

In [25]:
batch_size = 32
Input_shape = (30, 120, 120, 3)
model5 = Sequential()
model5.add(Conv3D(16, (2,2,2),input_shape=Input_shape,activation='relu'))
model5.add(BatchNormalization())
model5.add(MaxPooling3D(pool_size=(2,2,2)))
model5.add(Dropout(0.25))

model5.add(Conv3D(32, (2,2,2),activation='relu'))
model5.add(BatchNormalization())
model5.add(MaxPooling3D(pool_size=(2,2,2)))
model5.add(Dropout(0.25))

model5.add(Conv3D(64, (2,2,2),activation='relu'))
model5.add(BatchNormalization())
model5.add(MaxPooling3D(pool_size=(2,2,2)))
model5.add(Dropout(0.25))

model5.add(Flatten())

model5.add(Dense(512,activation='relu'))
#model.add(Activation('relu'))
model5.add(Dropout(0.4))


model5.add(Dense(5))
model5.add(Activation('softmax'))

model5.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model5.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 29, 119, 119, 16)  400       
_________________________________________________________________
batch_normalization_29 (Batc (None, 29, 119, 119, 16)  64        
_________________________________________________________________
max_pooling3d_29 (MaxPooling (None, 14, 59, 59, 16)    0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 14, 59, 59, 16)    0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 13, 58, 58, 32)    4128      
_________________________________________________________________
batch_normalization_30 (Batc (None, 13, 58, 58, 32)    128       
_________________________________________________________________
max_pooling3d_30 (MaxPooling (None, 6, 29, 29, 32)   

In [26]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [27]:
model5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 29.2400 - categorical_accuracy: 0.2971Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: val_loss improved from inf to 3.01275, saving model to model_init_2021-10-2704_53_32.764571/model-00001-29.24002-0.29713-3.01275-0.37000.h5
21/21 [==============================] - 207s 10s/step - loss: 29.2400 - categorical_accuracy: 0.2971 - val_loss: 3.0127 - val_categorical_accuracy: 0.3700
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 3.9825 - categorical_accuracy: 0.4676
Epoch 00002: val_loss did not improve from 3.01275
21/21 [==============================] - 204s 10s/step - loss: 3.9825 - categorical_accuracy: 0.4676 - val_loss: 8.7694 - val_categorical_accuracy: 0.2200
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 1.3673 - categorical_accuracy: 0.5415
Epoch 00003: val_loss did not impr

### Model 6

In [28]:
batch_size = 8

In [29]:
# 16,32,64 kernels in 3 consecutive conv3D layers
model6 = Sequential()
# 1st Layer Group
model6.add(Conv3D(16, kernel_size=(3,3,3), input_shape=(30,120,120,3),padding='same'))
model6.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(MaxPooling3D(pool_size=(3,3,3)))
# 2nd Layer Group
model6.add(Conv3D(32, kernel_size=(3,3,3),padding='same'))
model6.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(MaxPooling3D(pool_size=(3,3,3)))
model6.add(layers.Dropout(0.25))
# 3rd Layer Group
model6.add(Conv3D(64, kernel_size=(3,3,3),padding='same'))
model6.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(MaxPooling3D(pool_size=(3,3,3)))

# FC layer group
model6.add(Flatten())
model6.add(Dense(500, activation='relu'))
model6.add(layers.Dropout(0.5))
# Softmax Layer
model6.add(Dense(5, activation='softmax'))

In [30]:
model6.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model6.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_32 (Conv3D)           (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
batch_normalization_32 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
activation_11 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_32 (MaxPooling (None, 10, 40, 40, 16)    0         
_________________________________________________________________
conv3d_33 (Conv3D)           (None, 10, 40, 40, 32)    13856     
_________________________________________________________________
batch_normalization_33 (Batc (None, 10, 40, 40, 32)    128       
_________________________________________________________________
activation_12 (Activation)   (None, 10, 40, 40, 32)  

In [31]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [32]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [33]:
model6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 8
Epoch 1/20
83/83 [==============================] - ETA: 0s - loss: 2.8011 - categorical_accuracy: 0.2685Source path =  /home/datasets/Project_data/val ; batch size = 8

Epoch 00001: saving model to model_init_2021-10-2619_11_58.637805/model-00001-2.80113-0.26848-1.69528-0.26000.h5
83/83 [==============================] - 208s 3s/step - loss: 2.8011 - categorical_accuracy: 0.2685 - val_loss: 1.6953 - val_categorical_accuracy: 0.2600
Epoch 2/20
83/83 [==============================] - ETA: 0s - loss: 1.5047 - categorical_accuracy: 0.3514
Epoch 00002: saving model to model_init_2021-10-2619_11_58.637805/model-00002-1.50472-0.35143-2.04884-0.21000.h5
83/83 [==============================] - 203s 2s/step - loss: 1.5047 - categorical_accuracy: 0.3514 - val_loss: 2.0488 - val_categorical_accuracy: 0.2100
Epoch 3/20
83/83 [==============================] - ETA: 0s - loss: 1.3697 - categorical_accuracy: 0.4163
Epoch 00003: savin

#### The above model has been selected as the best model since it gives the best train and validation accuracy obtained at epoch 18 with 93.9% training accuracy and 89% validation accuracy.

### CNN + RNN Architecture

In [34]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed

#### Model 7 - Using LSTM

In [35]:
batch_size = 32

model7 = Sequential()

model7.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(30,120,120,3)))
model7.add(TimeDistributed(BatchNormalization()))
model7.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model7.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model7.add(TimeDistributed(BatchNormalization()))
model7.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model7.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model7.add(TimeDistributed(BatchNormalization()))
model7.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model7.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model7.add(TimeDistributed(BatchNormalization()))
model7.add(TimeDistributed(MaxPooling2D((2, 2))))
          

model7.add(TimeDistributed(Flatten()))

model7.add(Dropout(0.25))
model7.add(LSTM(64))
model7.add(Dropout(0.5))
        
        
model7.add(Dense(5, activation='softmax'))

In [36]:
model7.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model7.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 30, 30, 64)   

In [37]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [38]:
model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 1.5895 - categorical_accuracy: 0.2851Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: val_loss improved from inf to 1.92079, saving model to model_init_2021-10-2710_41_46.391063/model-00001-1.58954-0.28507-1.92079-0.19000.h5
21/21 [==============================] - 211s 10s/step - loss: 1.5895 - categorical_accuracy: 0.2851 - val_loss: 1.9208 - val_categorical_accuracy: 0.1900
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 1.4282 - categorical_accuracy: 0.4012
Epoch 00002: val_loss did not improve from 1.92079
21/21 [==============================] - 214s 10s/step - loss: 1.4282 - categorical_accuracy: 0.4012 - val_loss: 1.9315 - val_categorical_accuracy: 0.2600
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss:

### Model 8 - Using GRU

In [39]:
batch_size =64

model8 = Sequential()

model8.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(30,120,120,3)))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(MaxPooling2D((3, 3))))
        
model8.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(MaxPooling2D((3, 3))))
        
model8.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(MaxPooling2D((3, 3))))
                 

model8.add(TimeDistributed(Flatten()))


model8.add(GRU(64))
model8.add(Dropout(0.5))
        
        
model8.add(Dense(5, activation='softmax'))

In [40]:
model8.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model8.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 40, 40, 16)    0         
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 40, 40, 32)    4640      
_________________________________________________________________
time_distributed_17 (TimeDis (None, 30, 40, 40, 32)    128       
_________________________________________________________________
time_distributed_18 (TimeDis (None, 30, 13, 13, 32)    0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 30, 13, 13, 64)   

In [41]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [42]:
model8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/20
11/11 [==============================] - ETA: 0s - loss: 1.8567 - categorical_accuracy: 0.2911 Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: val_loss improved from inf to 1.59691, saving model to model_init_2021-10-2710_41_46.391063/model-00001-1.85671-0.29110-1.59691-0.23000.h5
11/11 [==============================] - 228s 21s/step - loss: 1.8567 - categorical_accuracy: 0.2911 - val_loss: 1.5969 - val_categorical_accuracy: 0.2300
Epoch 2/20
11/11 [==============================] - ETA: 0s - loss: 1.5086 - categorical_accuracy: 0.3906 
Epoch 00002: val_loss improved from 1.59691 to 1.53410, saving model to model_init_2021-10-2710_41_46.391063/model-00002-1.50860-0.39065-1.53410-0.34000.h5
11/11 [==============================] - 217s 20s/step - loss: 1.5086 - categorical_accuracy: 0.3906 - val_loss: 1.5341 - val_categorical_accuracy: 0.3400
Epoch 3/20
11/11 [==================

### Model 9 - Using ResNet50

In [43]:
#Use Transfer Learing ResNet50
from tensorflow.keras.applications import ResNet50

restnet_transfer = ResNet50(weights='imagenet', include_top=False,pooling='max')

#Make weight of the transer model trainable false
restnet_transfer.trainable = False

94773248/94765736 [==============================] - 3s 0us/step


In [44]:
batch_size = 64

model9 = Sequential()
model9.add(TimeDistributed(restnet_transfer,input_shape=(30,120,120,3)))

model9.add(GRU(64))
model9.add(Dropout(0.5))

model9.add(Dense(5, activation='softmax'))


In [45]:
model9.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model9.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_23 (TimeDis (None, 30, 2048)          23587712  
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                405888    
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
Total params: 23,993,925
Trainable params: 406,213
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [46]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [47]:
model9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/20
11/11 [==============================] - ETA: 0s - loss: 2.0402 - categorical_accuracy: 0.2006 Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: val_loss improved from inf to 1.57314, saving model to model_init_2021-10-2710_41_46.391063/model-00001-2.04016-0.20060-1.57314-0.32000.h5
11/11 [==============================] - 207s 19s/step - loss: 2.0402 - categorical_accuracy: 0.2006 - val_loss: 1.5731 - val_categorical_accuracy: 0.3200
Epoch 2/20
11/11 [==============================] - ETA: 0s - loss: 1.7752 - categorical_accuracy: 0.2413 
Epoch 00002: val_loss improved from 1.57314 to 1.46774, saving model to model_init_2021-10-2710_41_46.391063/model-00002-1.77522-0.24133-1.46774-0.49000.h5
11/11 [==============================] - 201s 18s/step - loss: 1.7752 - categorical_accuracy: 0.2413 - val_loss: 1.4677 - val_categorical_accuracy: 0.4900
Epoch 3/20
11/11 [==================

### Model 10 - Using MobileNet

In [48]:
#Use Transfer Learning MobileNet
from tensorflow.keras.applications import mobilenet

mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

#Make weight of the transer model trainable false
mobilenet_transfer.trainable = False

In [49]:
batch_size = 64

In [50]:
model10 = Sequential()
model10.add(TimeDistributed(mobilenet_transfer,input_shape=(30,120,120,3)))
        
model10.add(TimeDistributed(BatchNormalization()))
model10.add(TimeDistributed(MaxPooling2D((2, 2))))
model10.add(TimeDistributed(Flatten()))

model10.add(GRU(64))
model10.add(Dropout(0.5))
        
model10.add(Dense(64,activation='relu'))
model10.add(Dropout(0.5))
        
model10.add(Dense(5, activation='softmax'))

In [51]:
model10.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model10.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 1024)          0         
_________________________________________________________________
gru (GRU)                    (None, 64)                209280    
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)               

In [52]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, cooldown=1, verbose=1)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [53]:
model10.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/20
11/11 [==============================] - ETA: 0s - loss: 1.9778 - categorical_accuracy: 0.2428 Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: val_loss improved from inf to 1.54500, saving model to model_init_2021-10-2715_44_58.160471/model-00001-1.97778-0.24284-1.54500-0.24000.h5
11/11 [==============================] - 203s 18s/step - loss: 1.9778 - categorical_accuracy: 0.2428 - val_loss: 1.5450 - val_categorical_accuracy: 0.2400
Epoch 2/20
11/11 [==============================] - ETA: 0s - loss: 1.6003 - categorical_accuracy: 0.3047 
Epoch 00002: val_loss improved from 1.54500 to 1.46983, saving model to model_init_2021-10-2715_44_58.160471/model-00002-1.60028-0.30468-1.46983-0.44000.h5
11/11 [==============================] - 200s 18s/step - loss: 1.6003 - categorical_accuracy: 0.3047 - val_loss: 1.

### Best model is chosen as Model 6 (Conv3D) since it gives the best train and validation accuracy obtained at epoch 18 with 93.9% training accuracy and 89% validation accuracy.